In [0]:
import sys
sys.version

%reset -f
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
import sys
sys.version

#!pip3 install torch==0.4
# !pip3 install torchvision

!pip3 install 'torch==0.4.0'
!pip3 install 'torchvision==0.2.1'
!pip3 install --no-cache-dir -I 'pillow==5.1.0'
!pip3 install torchvision
print('done')
# Restart Kernel
# This workaround is needed to properly upgrade PIL on Google Colab.
import os
os._exit(0)

    100% |████████████████████████████████| 484.0MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b0f6000 @  0x7f9a197842a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 2.0MB 15.0MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 2.0MB 2.4MB/s 


In [1]:
import torch

print('__pyTorch VERSION:', torch.__version__)
use_cuda = torch.cuda.is_available()

print('use_cuda = {0}'.format(use_cuda))

from torch.utils.data import Dataset
from skimage import io, transform
from torch.utils.data import Dataset
import os
import pandas as pd
from PIL import Image
import random 
import numpy as np
SEED = 999

def fixSeed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

fixSeed(SEED)


print(1111)


__pyTorch VERSION: 0.4.0
use_cuda = True
1111


In [3]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gabib3b","key":"47a6ef15f64b324b1ecb96ea73bf1df7"}'}

In [0]:
!cp kaggle.json ~/.kaggle/


In [5]:
!kaggle datasets download -d vbookshelf/v2-plant-seedlings-dataset

 99% 1.59G/1.59G [00:40<00:00, 39.3MB/s]
100% 1.59G/1.59G [00:40<00:00, 42.1MB/s]


In [0]:
!unzip v2-plant-seedlings-dataset.zip -d data

Archive:  v2-plant-seedlings-dataset.zip
   creating: data/Black-grass/
  inflating: data/Black-grass/1.png  
  inflating: data/Black-grass/10.png  
  inflating: data/Black-grass/100.png  
  inflating: data/Black-grass/101.png  
  inflating: data/Black-grass/102.png  
  inflating: data/Black-grass/103.png  
  inflating: data/Black-grass/104.png  
  inflating: data/Black-grass/105.png  
  inflating: data/Black-grass/106.png  
  inflating: data/Black-grass/107.png  
  inflating: data/Black-grass/108.png  
  inflating: data/Black-grass/109.png  
  inflating: data/Black-grass/11.png  
  inflating: data/Black-grass/110.png  
  inflating: data/Black-grass/111.png  
  inflating: data/Black-grass/112.png  
  inflating: data/Black-grass/113.png  
  inflating: data/Black-grass/114.png  
  inflating: data/Black-grass/115.png  
  inflating: data/Black-grass/116.png  
  inflating: data/Black-grass/117.png  
  inflating: data/Black-grass/118.png  
  inflating: data/Black-grass/119.png  
  inflating: